In [167]:
from ocr import OCR
from PIL import Image
import cv2
import os
import keras_ocr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob as glob
import openpyxl as op
import re
from joblib import Parallel, delayed
import joblib
import uuid
import keras_ocr

import plotly.express as px

def flatten(l):
    return [item for sublist in l for item in sublist]

# Dictionary crops

In [236]:
crops={
    'densities':{},
    'faz':{}
      }
crops['densities']['table_top_left']={
                                'l' : 1530,
                                't' : 154,
                                'r' : 1600,
                                'b' : 540
                                }
crops['densities']['table_top_right']={
                                'l' : 1750,
                                't' : 154,
                                'r' : 1825,
                                'b' : 540
                                }
crops['densities']['table_bot']={
                                'l' : 1535,
                                't' : 560,
                                'r' : 1825,
                                'b' : 630
                                }
crops['densities']['quality']={
                                'l' : 1116,
                                't' : 55,
                                'r' : 1200,
                                'b' : 100
                                }
crops['faz']['values']={
                            'l' : 400,
                            't' : 90,
                            'r' : 800,
                            'b' : 125
                            }


crops['faz']['quality']={
                        'l' : 1116,
                        't' : 55,
                        'r' : 1200,
                        'b' : 100
                        }


# FILES

In [3]:
#first iteration
folders_patients=glob('../data/P*')
folders_visits=glob('../data/P*/V*')
visits=sorted(
    list(
        set([folder.split("/")[3] for folder in folders_visits])
    )
)
patients=sorted(
    list(
        set([folder.split('/')[2] for folder in folders_patients]) 
    )
)

#Calm down here
# imgs={}
# for patient in patients:
#     imgs[patient]={}
#     files=glob('../data/'+patient+"/*/*")
#     files=[file for file in files if 'Zone' not in file]
#     densities=[file for file in files if ('Density' in file) ]
#     fazs=[file for file in files if ('FAZ' in file)  ]
#     imgs[patient]['densities']=densities
#     imgs[patient]['fazs']=fazs


In [4]:
#second iteration
files=glob('../data/*/*/*png')
files=[file for file in files if 'QuickVue' not in file]

imgpervisit={}
for visit in visits:
    imgpervisit[visit]=sorted([file for file in files if visit in file])



In [5]:
pwd

'/home/ubuntu/Eyes/notebooks'

In [51]:
%%time
cols=[
 'QUALITY',
 'depth_whole',
 'depth_sup hemi',
 'depth_inf hemi',
 'depth_fovea',
 'depth_parafovea',
 'depth_para_sup hemi',
 'depth_parai_inf hemi',
 'depth_para temp',
 'depth_para_sup',
 'depth_para_nasal',
 'depth_para_inf',
 'depth_perifovea',
 'depth_peri_sup hemi',
 'depth_peri_inf hemi',
 'depth_peri_tempo ',
 'depth_peri_sup',
 'depth_peri_nasal',
 'depth_peri_inf',
 'thick_whole',
 'thick_sup hemi',
 'thick_inf hemi',
 'thick_fovea',
 'thick_parafovea',
 'thick_para_sup hemi',
 'thick_parai_inf hemi',
 'thick_para temp',
 'thick_para_sup',
 'thick_para_nasal',
 'thick_para_inf',
 'thick_perifovea',
 'thick_peri_sup hemi',
 'thick_peri_inf hemi',
 'thick_peri_tempo ',
 'thick_peri_sup',
 'thick_peri_nasal',
 'thick_peri_inf',
 'grid_v_d1',
 'grid_v_d2',
 'grid_v_d3',
 'grid_v_d4',
 'grid_v_d5',
 'grid_v_d6',
 'grid_v_d7',
 'grid_v_d8',
 'grid_v_d9',
 'area (mm2)',
 'perim (mm)',
 'FD']

errors_top=[]
errors_bot=[]
errors_qual=[]
error_faz=[]

df=pd.DataFrame(columns=['pat','v','eye','depth']+cols) # MAKE THE COLUMNS, cols come from readin the sheet.
for file in files:
    unique_id=uuid.uuid4()
    df=pd.DataFrame(columns=['pat','v','eye','depth']+cols) # MAKE THE COLUMNS, cols come from readin the sheet.

    print(file)
    pat=file.split('/')[2]
    v=file.split('/')[3]
    eye=file.split(',')[1].strip()
    depth=file.split(',')[3].split('-')[0].strip()
    unique_id=f"{pat}_{v}_{eye}_{depth}"+str(uuid.uuid4())
    os.mkdir(f'evaluate/{unique_id}')
#     fdelete = glob('evaluate/*')
#     for f in fdelete:
#         os.remove(f)
    if 'Density' in file:
        specification='SUP' if 'SUP' in file else 'DP'
        im = Image.open(file)

        cropped_top1= im.crop((crops['densities']['table_top_left']['l'],
                              crops['densities']['table_top_left']['t'],
                              crops['densities']['table_top_left']['r'],
                              crops['densities']['table_top_left']['b']))
        cropped_top2= im.crop((crops['densities']['table_top_right']['l'],
                              crops['densities']['table_top_right']['t'],
                              crops['densities']['table_top_right']['r'],
                              crops['densities']['table_top_right']['b']))
        
        cropped_bot= im.crop((crops['densities']['table_bot']['l'],
                              crops['densities']['table_bot']['t'],
                              crops['densities']['table_bot']['r'],
                              crops['densities']['table_bot']['b']))
        cropped_qual= im.crop((crops['densities']['quality']['l'],
                               crops['densities']['quality']['t'],
                               crops['densities']['quality']['r'],
                               crops['densities']['quality']['b']))
        cropped_top1.save(f'evaluate/{unique_id}/a_cropped_top1.png')
        cropped_top2.save(f'evaluate/{unique_id}/b_cropped_top2.png')
        cropped_qual.save(f'evaluate/{unique_id}/c_cropped_quality.png')
        cropped_bot.save(f'evaluate/{unique_id}/d_cropped_bot.png')
        
        ocr=OCR(image_folder=f"evaluate/{unique_id}/")
        results=ocr.easyocr_model_works(visualization=False)  
        temp={   'QUALITY': np.nan,
                 'depth_whole': np.nan,
                 'depth_sup hemi': np.nan,
                 'depth_inf hemi': np.nan,
                 'depth_fovea': np.nan,
                 'depth_parafovea': np.nan,
                 'depth_para_sup hemi': np.nan,
                 'depth_parai_inf hemi': np.nan,
                 'depth_para temp': np.nan,
                 'depth_para_sup': np.nan,
                 'depth_para_nasal': np.nan,
                 'depth_para_inf': np.nan,
                 'depth_perifovea': np.nan,
                 'depth_peri_sup hemi': np.nan,
                 'depth_peri_inf hemi': np.nan,
                 'depth_peri_tempo ': np.nan,
                 'depth_peri_sup': np.nan,
                 'depth_peri_nasal': np.nan,
                 'depth_peri_inf': np.nan,
                 'thick_whole': np.nan,
                 'thick_sup hemi': np.nan,
                 'thick_inf hemi': np.nan,
                 'thick_fovea': np.nan,
                 'thick_parafovea': np.nan,
                 'thick_para_sup hemi': np.nan,
                 'thick_parai_inf hemi': np.nan,
                 'thick_para temp': np.nan,
                 'thick_para_sup': np.nan,
                 'thick_para_nasal': np.nan,
                 'thick_para_inf': np.nan,
                 'thick_perifovea': np.nan,
                 'thick_peri_sup hemi': np.nan,
                 'thick_peri_inf hemi': np.nan,
                 'thick_peri_tempo ': np.nan,
                 'thick_peri_sup': np.nan,
                 'thick_peri_nasal': np.nan,
                 'thick_peri_inf': np.nan,
                 'grid_v_d1': np.nan,
                 'grid_v_d2': np.nan,
                 'grid_v_d3': np.nan,
                 'grid_v_d4': np.nan,
                 'grid_v_d5': np.nan,
                 'grid_v_d6': np.nan,
                 'grid_v_d7': np.nan,
                 'grid_v_d8': np.nan,
                 'grid_v_d9':np.nan
                }
##################################################################################QUALITY
        try:
            qual=results[2][0][1]
            temp[list(temp.keys())[0]]=qual
        except Exception as e:
            errors_qual.append(['quality_error in',file,e,])
            cropped_top1= im.crop((crops['densities']['table_top_left']['l'],
                                  crops['densities']['table_top_left']['t']+215,
                                  crops['densities']['table_top_left']['r'],
                                  crops['densities']['table_top_left']['b']+215))
            cropped_top2= im.crop((crops['densities']['table_top_right']['l'],
                                  crops['densities']['table_top_right']['t']+215,
                                  crops['densities']['table_top_right']['r'],
                                  crops['densities']['table_top_right']['b']+215))

            cropped_bot= im.crop((crops['densities']['table_bot']['l'],
                                  crops['densities']['table_bot']['t']+215,
                                  crops['densities']['table_bot']['r'],
                                  crops['densities']['table_bot']['b']+215))
            cropped_qual= im.crop((crops['densities']['quality']['l'],
                                   crops['densities']['quality']['t']+215,
                                   crops['densities']['quality']['r'],
                                   crops['densities']['quality']['b']+215))
            fdelete = glob(f'evaluate/{unique_id}/*')
            for f in fdelete:
                os.remove(f)
            cropped_top1.save(f'evaluate/{unique_id}/a_cropped_top1.png')
            cropped_top2.save(f'evaluate/{unique_id}/b_cropped_top2.png')
            cropped_qual.save(f'evaluate/{unique_id}/c_cropped_quality.png')
            cropped_bot.save(f'evaluate/{unique_id}/d_cropped_bot.png')
            ocr=OCR(image_folder=f"evaluate/{unique_id}/")
            results=ocr.easyocr_model_works(visualization=False)  
            try:
                qual=results[2][0][1]
                temp[list(temp.keys())[0]]=qual
            except Exception as ee:
                errors_qual.append(['Definite quality error in ',file,ee])
##################################################################################TABLE TOP
        try:
            data_2d=zip(np.array(results[0],dtype=object)[:,1],np.array(results[1],dtype=object)[:,1])
            df_top=pd.DataFrame(data_2d)
            for i,val in enumerate(df_top[0].values,start=1):
                temp[list(temp.keys())[i]]=val
            for i,val in enumerate(df_top[1].values,start=19):
                temp[list(temp.keys())[i]]=val
        except Exception as e:
            errors_top.append(['top_error in',file,e])
            
##################################################################################TABLE BOT
        data_lin=np.array(results[3],dtype=object)[:,1]
        try:
            data_2d=np.split(data_lin,data_lin.shape[0]/3)
            df_bot=pd.DataFrame(data_2d)
            df_bot_list=flatten(df_bot.values.tolist())
            for i,val in enumerate(df_bot_list,start=37):
                temp[list(temp.keys())[i]]=val
            temp['pat']  = pat
            temp['v']    = v
            temp['eye']  = eye
            temp['depth']= depth
            temp['specification']=specification
            df=df.append(
                temp,
                ignore_index=True)
        except Exception as e:
            errors_bot.append(['bot_error in',file,e])
#####################################################################
#####################################################################  FAZ
#####################################################################
    elif 'FAZ' in file:
        specification='FAZ'
        im = Image.open(file)
        cropped_values= im.crop((crops['faz']['values']['l'],
                                 crops['faz']['values']['t'],
                                 crops['faz']['values']['r'],
                                 crops['faz']['values']['b']))

        cropped_qual= im.crop((crops['faz']['quality']['l'],
                               crops['faz']['quality']['t'],
                               crops['faz']['quality']['r'],
                               crops['faz']['quality']['b']))
        cropped_values.save(f'evaluate/{unique_id}/a_cropped_values.png')
        cropped_qual.save(f'evaluate/{unique_id}/b_cropped_quality.png')
        ocr=OCR(image_folder=f"evaluate/{unique_id}/")
        results=ocr.easyocr_model_works(visualization=False)
        try:
            temp=string=' '.join([result[1] for result in results[0]]) #extracting all the ugly strings from OCR for first 3 values
            clean_3=re.findall(r"[-+]?(?:\d*\.\d+|\d+)", temp)         #regex to extract only the numbers
            last=string=' '.join([result[1] for result in results[1]]) #extract quality
            if type(last) is not list:
                last=[last]
            res_clean=clean_3+last

            df=df.append({
                'pat':pat,
                'v':v,
                'eye':eye,
                'depth':depth,
                'area (mm2)':res_clean[0],
                'perim (mm)':res_clean[1],
                'FD':res_clean[2],
                'QUALITY':res_clean[-1],
                'specification':specification
            },
            ignore_index=True)
        except Exception as e:
            error_faz.append(['quality_error in',file,e])


../data/P002/V3/P002, R eye, V3, 6.0 - FAZ.png
Looking for /home/ubuntu/.keras-ocr/craft_mlt_25k.h5


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Looking for /home/ubuntu/.keras-ocr/crnn_kurapan.h5


<timed exec>:237: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


../data/P002/V3/P002, R eye, V3, 3.0 - Density DP.png
Looking for /home/ubuntu/.keras-ocr/craft_mlt_25k.h5


...

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Looking for /home/ubuntu/.keras-ocr/crnn_kurapan.h5
Looking for /home/ubuntu/.keras-ocr/craft_mlt_25k.h5


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Looking for /home/ubuntu/.keras-ocr/crnn_kurapan.h5


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## JOBLIB

In [237]:
%%time
cols=[
 'QUALITY',
 'depth_whole',
 'depth_sup hemi',
 'depth_inf hemi',
 'depth_fovea',
 'depth_parafovea',
 'depth_para_sup hemi',
 'depth_parai_inf hemi',
 'depth_para temp',
 'depth_para_sup',
 'depth_para_nasal',
 'depth_para_inf',
 'depth_perifovea',
 'depth_peri_sup hemi',
 'depth_peri_inf hemi',
 'depth_peri_tempo ',
 'depth_peri_sup',
 'depth_peri_nasal',
 'depth_peri_inf',
 'thick_whole',
 'thick_sup hemi',
 'thick_inf hemi',
 'thick_fovea',
 'thick_parafovea',
 'thick_para_sup hemi',
 'thick_parai_inf hemi',
 'thick_para temp',
 'thick_para_sup',
 'thick_para_nasal',
 'thick_para_inf',
 'thick_perifovea',
 'thick_peri_sup hemi',
 'thick_peri_inf hemi',
 'thick_peri_tempo ',
 'thick_peri_sup',
 'thick_peri_nasal',
 'thick_peri_inf',
 'grid_v_d1',
 'grid_v_d2',
 'grid_v_d3',
 'grid_v_d4',
 'grid_v_d5',
 'grid_v_d6',
 'grid_v_d7',
 'grid_v_d8',
 'grid_v_d9',
 'area (mm2)',
 'perim (mm)',
 'FD']

errors_top=[]
errors_bot=[]
errors_qual=[]
error_faz=[]

    
def extract(file):    
    unique_id=uuid.uuid4()
    df=pd.DataFrame(columns=['pat','v','eye','depth']+cols) # MAKE THE COLUMNS, cols come from readin the sheet.

    print(file)
    pat=file.split('/')[2]
    v=file.split('/')[3]
    eye=file.split(',')[1].strip()
    depth=file.split(',')[3].split('-')[0].strip()
    unique_id=f"{pat}_{v}_{eye}_{depth}"+str(uuid.uuid4())
    os.mkdir(f'evaluate/{unique_id}')
#     fdelete = glob('evaluate/*')
#     for f in fdelete:
#         os.remove(f)
    if 'Density' in file:
        specification='SUP' if 'SUP' in file else 'DP'
        im = Image.open(file)

        cropped_top1= im.crop((crops['densities']['table_top_left']['l'],
                              crops['densities']['table_top_left']['t'],
                              crops['densities']['table_top_left']['r'],
                              crops['densities']['table_top_left']['b']))
        cropped_top2= im.crop((crops['densities']['table_top_right']['l'],
                              crops['densities']['table_top_right']['t'],
                              crops['densities']['table_top_right']['r'],
                              crops['densities']['table_top_right']['b']))
        
        cropped_bot= im.crop((crops['densities']['table_bot']['l'],
                              crops['densities']['table_bot']['t'],
                              crops['densities']['table_bot']['r'],
                              crops['densities']['table_bot']['b']))
        cropped_qual= im.crop((crops['densities']['quality']['l'],
                               crops['densities']['quality']['t'],
                               crops['densities']['quality']['r'],
                               crops['densities']['quality']['b']))
        cropped_top1.save(f'evaluate/{unique_id}/a_cropped_top1.png')
        cropped_top2.save(f'evaluate/{unique_id}/b_cropped_top2.png')
        cropped_qual.save(f'evaluate/{unique_id}/c_cropped_quality.png')
        cropped_bot.save(f'evaluate/{unique_id}/d_cropped_bot.png')
        
        ocr=OCR(image_folder=f"evaluate/{unique_id}/")
        results=ocr.easyocr_model_works(visualization=False)  
        temp={   'QUALITY': np.nan,
                 'depth_whole': np.nan,
                 'depth_sup hemi': np.nan,
                 'depth_inf hemi': np.nan,
                 'depth_fovea': np.nan,
                 'depth_parafovea': np.nan,
                 'depth_para_sup hemi': np.nan,
                 'depth_parai_inf hemi': np.nan,
                 'depth_para temp': np.nan,
                 'depth_para_sup': np.nan,
                 'depth_para_nasal': np.nan,
                 'depth_para_inf': np.nan,
                 'depth_perifovea': np.nan,
                 'depth_peri_sup hemi': np.nan,
                 'depth_peri_inf hemi': np.nan,
                 'depth_peri_tempo ': np.nan,
                 'depth_peri_sup': np.nan,
                 'depth_peri_nasal': np.nan,
                 'depth_peri_inf': np.nan,
                 'thick_whole': np.nan,
                 'thick_sup hemi': np.nan,
                 'thick_inf hemi': np.nan,
                 'thick_fovea': np.nan,
                 'thick_parafovea': np.nan,
                 'thick_para_sup hemi': np.nan,
                 'thick_parai_inf hemi': np.nan,
                 'thick_para temp': np.nan,
                 'thick_para_sup': np.nan,
                 'thick_para_nasal': np.nan,
                 'thick_para_inf': np.nan,
                 'thick_perifovea': np.nan,
                 'thick_peri_sup hemi': np.nan,
                 'thick_peri_inf hemi': np.nan,
                 'thick_peri_tempo ': np.nan,
                 'thick_peri_sup': np.nan,
                 'thick_peri_nasal': np.nan,
                 'thick_peri_inf': np.nan,
                 'grid_v_d1': np.nan,
                 'grid_v_d2': np.nan,
                 'grid_v_d3': np.nan,
                 'grid_v_d4': np.nan,
                 'grid_v_d5': np.nan,
                 'grid_v_d6': np.nan,
                 'grid_v_d7': np.nan,
                 'grid_v_d8': np.nan,
                 'grid_v_d9':np.nan
                }
##################################################################################QUALITY
        try:
            qual=results[2][0][1]
            temp[list(temp.keys())[0]]=qual
        except Exception as e:
            errors_qual.append(['quality_error in',file,e,])
            cropped_top1= im.crop((crops['densities']['table_top_left']['l'],
                                  crops['densities']['table_top_left']['t']+215,
                                  crops['densities']['table_top_left']['r'],
                                  crops['densities']['table_top_left']['b']+215))
            cropped_top2= im.crop((crops['densities']['table_top_right']['l'],
                                  crops['densities']['table_top_right']['t']+215,
                                  crops['densities']['table_top_right']['r'],
                                  crops['densities']['table_top_right']['b']+215))

            cropped_bot= im.crop((crops['densities']['table_bot']['l'],
                                  crops['densities']['table_bot']['t']+215,
                                  crops['densities']['table_bot']['r'],
                                  crops['densities']['table_bot']['b']+215))
            cropped_qual= im.crop((crops['densities']['quality']['l'],
                                   crops['densities']['quality']['t']+215,
                                   crops['densities']['quality']['r'],
                                   crops['densities']['quality']['b']+215))
            fdelete = glob(f'evaluate/{unique_id}/*')
            for f in fdelete:
                os.remove(f)
            cropped_top1.save(f'evaluate/{unique_id}/a_cropped_top1.png')
            cropped_top2.save(f'evaluate/{unique_id}/b_cropped_top2.png')
            cropped_qual.save(f'evaluate/{unique_id}/c_cropped_quality.png')
            cropped_bot.save(f'evaluate/{unique_id}/d_cropped_bot.png')
            ocr=OCR(image_folder=f"evaluate/{unique_id}/")
            results=ocr.easyocr_model_works(visualization=False)  
            try:
                qual=results[2][0][1]
                temp[list(temp.keys())[0]]=qual
            except Exception as ee:
                errors_qual.append(['Definite quality error in ',file,ee])
                
##################################################################################TABLE TOP
        try:
            data_2d=zip(np.array(results[0],dtype=object)[:,1],np.array(results[1],dtype=object)[:,1])
            df_top=pd.DataFrame(data_2d)
            for i,val in enumerate(df_top[0].values,start=1):
                temp[list(temp.keys())[i]]=val
            for i,val in enumerate(df_top[1].values,start=19):
                temp[list(temp.keys())[i]]=val
        except Exception as e:
            errors_top.append(['top_error in',file,e])
            
##################################################################################TABLE BOT
        data_lin=np.array(results[3],dtype=object)[:,1]
        try:
            data_2d=np.split(data_lin,data_lin.shape[0]/3)
            df_bot=pd.DataFrame(data_2d)
            df_bot_list=flatten(df_bot.values.tolist())
            for i,val in enumerate(df_bot_list,start=37):
                temp[list(temp.keys())[i]]=val
            temp['pat']  = pat
            temp['v']    = v
            temp['eye']  = eye
            temp['depth']= depth
            temp['specification']=specification
            df=df.append(
                temp,
                ignore_index=True)
        except Exception as e:
            errors_bot.append(['bot_error in',file,e])
#####################################################################
#####################################################################  FAZ
#####################################################################
    elif 'FAZ' in file:
        specification='FAZ'
        im = Image.open(file)
        cropped_values= im.crop((crops['faz']['values']['l'],
                                 crops['faz']['values']['t'],
                                 crops['faz']['values']['r'],
                                 crops['faz']['values']['b']))

        cropped_qual= im.crop((crops['faz']['quality']['l'],
                               crops['faz']['quality']['t'],
                               crops['faz']['quality']['r'],
                               crops['faz']['quality']['b']))
        cropped_values.save(f'evaluate/{unique_id}/a_cropped_values.png')
        cropped_qual.save(f'evaluate/{unique_id}/b_cropped_quality.png')
        ocr=OCR(image_folder=f"evaluate/{unique_id}/")
        results=ocr.easyocr_model_works(visualization=False)
        try:
            temp=string=' '.join([result[1] for result in results[0]]) #extracting all the ugly strings from OCR for first 3 values
            clean_3=re.findall(r"[-+]?(?:\d*\.\d+|\d+)", temp)         #regex to extract only the numbers
            last=string=' '.join([result[1] for result in results[1]]) #extract quality
            if type(last) is not list:
                last=[last]
            res_clean=clean_3+last

            df=df.append({
                'pat':pat,
                'v':v,
                'eye':eye,
                'depth':depth,
                'area (mm2)':res_clean[0],
                'perim (mm)':res_clean[1],
                'FD':res_clean[2],
                'QUALITY':res_clean[-1],
                'specification':specification
            },
            ignore_index=True)
        except Exception as e:
            error_faz.append(['quality_error in',file,e])
    return df

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 19.1 µs


In [238]:
%%time
temp=Parallel(n_jobs=-1)(delayed(extract)(file)for file in files)

2022-11-30 04:51:05.843396: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 04:51:06.188372: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 04:51:06.224710: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

2022-11-30 04:51:07.304800: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ubuntu/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:/home/ubuntu/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-11-30 04:51:07.304876: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ubuntu/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:/home/ubuntu/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-11-30 04:51:07.304883: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make 

2022-11-30 04:51:10.813048: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-30 04:51:10.813104: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubp): /proc/driver/nvidia/version does not exist
2022-11-30 04:51:10.813624: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 04:51:10.849647: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-30 04:51:10.849706: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] k

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
<timed exec>:241: FutureWarning: The frame.append method is depre

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
<timed exec>:241: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:241: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:209: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:241: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:241: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:209: FutureWarning: The frame.append method is deprecated and 

...

<timed exec>:241: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
2022-11-30 04:51:51.038079: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-30 04:51:51.038169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubp): /proc/driver/nvidia/version does not exist
2022-11-30 04:51:51.038937: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
<timed exec>:209: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

../data/P002/V3/P002, R eye, V3, 3.0 - FAZ.png
Looking for /home/ubuntu/.keras-ocr/craft_mlt_25k.h5
Looking for /home/ubuntu/.keras-ocr/crnn_kurapan.h5
../data/P002/V2/P002, R eye, V2, 3.0 - Density DP.png
Looking for /home/ubuntu/.keras-ocr/craft_mlt_25k.h5
Looking for /home/ubuntu/.keras-ocr/crnn_kurapan.h5
../data/P001/V2/P001, L eye, V2, 3.0 - FAZ.png
Looking for /home/ubuntu/.keras-ocr/craft_mlt_25k.h5
Looking for /home/ubuntu/.keras-ocr/crnn_kurapan.h5
../data/P006/V3/P006, R eye, V3, 3.0 - FAZ.png
Looking for /home/ubuntu/.keras-ocr/craft_mlt_25k.h5
Looking for /home/ubuntu/.keras-ocr/crnn_kurapan.h5
../data/P006/V2/P006, L eye, V2, 6.0 - Density DP.png
Looking for /home/ubuntu/.keras-ocr/craft_mlt_25k.h5
Looking for /home/ubuntu/.keras-ocr/crnn_kurapan.h5
../data/P003/V3/P003, R eye, V3, 3.0 - Density DP.png
Looking for /home/ubuntu/.keras-ocr/craft_mlt_25k.h5
Looking for /home/ubuntu/.keras-ocr/crnn_kurapan.h5
../data/P003/V2/P003, L eye, V2, 3.0 - Density SUP.png
Looking for 

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
<timed exec>:209: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:241: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:241: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


CPU times: user 904 ms, sys: 1.49 s, total: 2.39 s
Wall time: 52.6 s


<timed exec>:209: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


## CLEAN DF

In [239]:
df=pd.concat(temp)
for col in df.columns:
    try:
        df[col]=df[col].str.replace(",",".",)
        df[col]=df[col].str.replace("\.\.",".",)
        df=df.replace('NJA',np.nan)
        df=df.replace('N/A',np.nan)
        
    except:
        pass
df

/tmp/ipykernel_261305/3932637153.py:5: FutureWarning:

The default value of regex will change from True to False in a future version.



,pat,v,eye,depth,QUALITY,depth_whole,depth_sup hemi,depth_inf hemi,depth_fovea,depth_parafovea,...,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9,area (mm2),perim (mm),FD,specification
0,P002,V3,R eye,6.0,9/10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.396,2.407,57.54,FAZ
0,P002,V3,R eye,3.0,9/10,51.2,51.9,50.6,23.5,534,...,53.7,31.0,50.7,53.5,52.3,53.8,NaN,NaN,NaN,DP
0,P002,V3,L eye,3.0,9/10,47.2,47.3,47.1,14.5,50.5,...,51.2,21.9,453,533,51.6,48.6,NaN,NaN,NaN,SUP
0,P002,V3,R eye,6.0,9/10,52.9,51.4,54.3,27.9,56.6,...,56.8,52.8,55.6,56.9,54.5,51.0,NaN,NaN,NaN,DP
0,P002,V3,L eye,6.0,9/10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.358,2.359,57.19,FAZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,P003,V2,L eye,3.0,6/10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.282,2044,46.12,FAZ
0,P003,V2,R eye,3.0,8/10,53.2,53.5,53.0,36.9,54.9,...,56.0,41.5,56.0,53.6,55.1,55.4,NaN,NaN,NaN,DP
0,P003,V2,R eye,6.0,8/10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.254,1.911,53.52,FAZ
0,P003,V2,L eye,6.0,5/10,40.33,40.4,40.3,14.4,39.2,...,42.9,35.5,36.6,48.0,40.0,36.4,NaN,NaN,NaN,SUP


# PIVOT DF

In [313]:
#kinda works but ugly 
# df.iloc[0].fillna('')+df.iloc[1].fillna('')

#THIS IS THE SOLUTION. NP.NAN != NP.NAN , WHEREAS EVERYTHING ELSE == TO ITSELF. THEREFORE I USE THAT TO DISCRIMINATE
s='DP' #AGGREGATE EVERYTHING THAT IS NOT DP, so really SP
df_pivot_sup=pd.pivot_table(
    df[(df.specification!=s)],
    index=["pat",'v','depth','eye'],
    aggfunc=lambda x: x.iloc[0] if(x.iloc[0]==x.iloc[0] ) or (len(x)==1) else x.iloc[1]
              )
df_pivot_sup=df_pivot_sup.apply(pd.to_numeric,errors='ignore')
df_pivot_sup=df_pivot_sup.drop(['specification'],axis=1)

s='SUP' #AGGREGATE EVERYTHING THAT IS NOT DP, so really SP
df_pivot_dp=pd.pivot_table(
    df[(df.specification!=s)],
    index=["pat",'v','depth','eye'],
    aggfunc=lambda x: x.iloc[0] if(x.iloc[0]==x.iloc[0] ) or (len(x)==1) else x.iloc[1]
              )
df_pivot_dp=df_pivot_dp.apply(pd.to_numeric,errors='ignore')
df_pivot_dp=df_pivot_dp.drop(['specification'],axis=1)

In [317]:
df_pivot_sup.to_csv('df_pivot_SUP.csv')
df_pivot_dp.to_csv('df_pivot_DP.csv')

## OUTLIERS

In [253]:
from scipy import stats

# df_pivot[(np.abs(stats.zscore(df_pivot.drop(['QUALITY', 'specification'],axis=1))) > 3).any(axis=1)]
dfn=df_pivot.drop(['QUALITY', 'specification'],axis=1)
cols=dfn.columns
fin=[]
for col in cols:
    zscore=stats.zscore(dfn[col]) > 3
    if zscore.any():
#         fin.append([col,zscore[zscore==True].index])
        fin.append([col,df_pivot.loc[zscore[zscore==True].index]])

In [278]:
i=10
print(fin[i][1].index,fin[i][0])
df_pivot.loc[fin[i][1].index]

MultiIndex([('P006', 'V3', '6.0', 'R eye')],
           names=['pat', 'v', 'depth', 'eye']) grid_v_d1


,,,,FD,QUALITY,area (mm2),depth_fovea,depth_inf hemi,depth_para temp,depth_para_inf,depth_para_nasal,depth_para_sup,depth_para_sup hemi,depth_parafovea,depth_parai_inf hemi,depth_peri_inf,depth_peri_inf hemi,depth_peri_nasal,depth_peri_sup,depth_peri_sup hemi,depth_peri_tempo,depth_perifovea,depth_sup hemi,depth_whole,grid_v_d1,grid_v_d2,grid_v_d3,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9,perim (mm),specification,thick_fovea,thick_inf hemi,thick_para temp,thick_para_inf,thick_para_nasal,thick_para_sup,thick_para_sup hemi,thick_parafovea,thick_parai_inf hemi,thick_peri_inf,thick_peri_inf hemi,thick_peri_nasal,thick_peri_sup,thick_peri_sup hemi,thick_peri_tempo,thick_perifovea,thick_sup hemi,thick_whole
pat,v,depth,eye,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
P006,V3,6.0,R eye,51.4,7/10,0.282,11.4,42.5,47.7,43.9,42.5,50.9,47.8,46.2,44.7,43.8,43.3,48.7,46.1,45.2,38.6,44.2,44.0,433.0,3644.0,46.6,50.2,40.5,40.2,46.8,38.1,41.8,49.0,2.035,FAZ,248,260,299,311,315,310,308,309,309,254.0,258.0,282.0,263.0,264.0,247.0,261.0,268,264


In [277]:
pd.set_option('display.max_columns', 60)

# DF COLLATION

## FAZ V2 &V3

In [714]:
cols=['QUALITY','area (mm2)','perim (mm)','FD']

a_faz_v2=df_pivot_dp.xs('V2',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=lambda x:x).xs('L eye',level=1,axis=1
                                                                 ).xs('3.0',level=1,axis=1)[cols]
b_faz_v2=df_pivot_dp.xs('V2',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=lambda x:x).xs('L eye',level=1,axis=1
                                                                 ).xs('6.0',level=1,axis=1)[cols]
c_faz_v2=df_pivot_dp.xs('V2',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=lambda x:x).xs('R eye',level=1,axis=1
                                                                 ).xs('3.0',level=1,axis=1)[cols]
d_faz_v2=df_pivot_dp.xs('V2',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=lambda x:x).xs('R eye',level=1,axis=1
                                                                 ).xs('6.0',level=1,axis=1)[cols]
faz_v2=pd.concat([a_faz_v2,b_faz_v2,c_faz_v2,d_faz_v2],axis=1)
faz_v2=faz_v2.reset_index()
faz_v2

a_faz_v3=df_pivot_dp.xs('V3',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=lambda x:x).xs('L eye',level=1,axis=1
                                                                 ).xs('3.0',level=1,axis=1)[cols]
b_faz_v3=df_pivot_dp.xs('V3',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=lambda x:x).xs('L eye',level=1,axis=1
                                                                 ).xs('6.0',level=1,axis=1)[cols]
c_faz_v3=df_pivot_dp.xs('V3',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=lambda x:x).xs('R eye',level=1,axis=1
                                                                 ).xs('3.0',level=1,axis=1)[cols]
d_faz_v3=df_pivot_dp.xs('V3',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=lambda x:x).xs('R eye',level=1,axis=1
                                                                 ).xs('6.0',level=1,axis=1)[cols]
faz_v3=pd.concat([a_faz_v3,b_faz_v3,c_faz_v3,d_faz_v3],axis=1)
faz_v3=faz_v3.reset_index()
faz_v3

,pat,QUALITY,area (mm2),perim (mm),FD,QUALITY,area (mm2),perim (mm),FD,QUALITY,area (mm2),perim (mm),FD,QUALITY,area (mm2),perim (mm),FD
0,P001,8/10,0.272,2110.000,49.05,8/10,0.238,1.900,54.17,8/10,0.232,1.864,50.48,9/10,0.209,1.741,52.02
1,P002,9/10,0.379,2610.000,54.18,NaN,NaN,NaN,NaN,9/10,0.401,2447.000,52.59,9/10,-0.380,2.347,55.74
2,P003,6/10,0.282,2044.000,46.12,5/10,0.275,1.953,43.89,8/10,0.261,2001.000,51.50,8/10,0.254,1.911,53.52
3,P006,7/10,-0.309,2.231,50.87,6/10,0.297,2.081,52.15,8/10,-0.286,2102.000,47.37,8/10,0.279,2.043,56.55


,pat,QUALITY,area (mm2),perim (mm),FD,QUALITY,area (mm2),perim (mm),FD,QUALITY,area (mm2),perim (mm),FD,QUALITY,area (mm2),perim (mm),FD
0,P001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/10,0.222,1.825,51.96,8/10,0.213,1.750,54.65
1,P002,9/10,0.371,2.506,54.70,9/10,-0.358,2.359,57.19,9/10,0.398,2.462,53.04,9/10,-0.396,2.407,57.54
2,P003,9/10,-0.293,2173.000,54.12,8/10,0.278,1.983,54.65,9/10,0.269,989.000,53.36,9/10,0.253,1.907,57.48
3,P006,7/10,-0.296,2118.000,49.00,7/10,0.288,2.056,53.61,8/10,-0.290,2.000,108.00,7/10,0.282,2.035,51.40


# DENSITY DP DATA

In [638]:
cols=[
 'QUALITY',
 'depth_whole',
 'depth_sup hemi',
 'depth_inf hemi',
 'depth_fovea',
 'depth_parafovea',
 'depth_para_sup hemi',
 'depth_parai_inf hemi',
 'depth_para temp',
 'depth_para_sup',
 'depth_para_nasal',
 'depth_para_inf',
 'depth_perifovea',
 'depth_peri_sup hemi',
 'depth_peri_inf hemi',
 'depth_peri_tempo ',
 'depth_peri_sup',
 'depth_peri_nasal',
 'depth_peri_inf',
 'thick_whole',
 'thick_sup hemi',
 'thick_inf hemi',
 'thick_fovea',
 'thick_parafovea',
 'thick_para_sup hemi',
 'thick_parai_inf hemi',
 'thick_para temp',
 'thick_para_sup',
 'thick_para_nasal',
 'thick_para_inf',
 'thick_perifovea',
 'thick_peri_sup hemi',
 'thick_peri_inf hemi',
 'thick_peri_tempo ',
 'thick_peri_sup',
 'thick_peri_nasal',
 'thick_peri_inf',
 'grid_v_d1',
 'grid_v_d2',
 'grid_v_d3',
 'grid_v_d4',
 'grid_v_d5',
 'grid_v_d6',
 'grid_v_d7',
 'grid_v_d8',
 'grid_v_d9',
]

## V2

In [618]:
dp_v2_piv=df_pivot_dp[cols].xs('V2',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=list )

dp_v2_piv=dp_v2_piv.applymap(lambda x: x[0] if type(x)==list else x)

a_dp_v2=dp_v2_piv.xs('L eye',level=1,axis=1).xs('3.0',level=1,axis=1)
b_dp_v2=dp_v2_piv.xs('L eye',level=1,axis=1).xs('6.0',level=1,axis=1)
c_dp_v2=dp_v2_piv.xs('R eye',level=1,axis=1).xs('3.0',level=1,axis=1)
d_dp_v2=dp_v2_piv.xs('R eye',level=1,axis=1).xs('6.0',level=1,axis=1)
dp_v2=pd.concat([a_dp_v2[cols],b_dp_v2[cols],c_dp_v2[cols],d_dp_v2[cols]],axis=1)
dp_v2=dp_v2.reset_index()
dp_v2

,pat,QUALITY,depth_whole,depth_sup hemi,depth_inf hemi,depth_fovea,depth_parafovea,depth_para_sup hemi,depth_parai_inf hemi,depth_para temp,depth_para_sup,depth_para_nasal,depth_para_inf,depth_perifovea,depth_peri_sup hemi,depth_peri_inf hemi,depth_peri_tempo,depth_peri_sup,depth_peri_nasal,depth_peri_inf,thick_whole,thick_sup hemi,thick_inf hemi,thick_fovea,thick_parafovea,thick_para_sup hemi,thick_parai_inf hemi,thick_para temp,thick_para_sup,thick_para_nasal,...,depth_peri_sup,depth_peri_nasal,depth_peri_inf,thick_whole,thick_sup hemi,thick_inf hemi,thick_fovea,thick_parafovea,thick_para_sup hemi,thick_parai_inf hemi,thick_para temp,thick_para_sup,thick_para_nasal,thick_para_inf,thick_perifovea,thick_peri_sup hemi,thick_peri_inf hemi,thick_peri_tempo,thick_peri_sup,thick_peri_nasal,thick_peri_inf,grid_v_d1,grid_v_d2,grid_v_d3,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9
0,P001,8/10,53.8,54.1,53.5,32.9,55.6,55.6,55.7,54.3,56.3,54.2,57.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307,308,306,267,319,321,316,309,323,324,...,52.0,49.5,5533.0,276,280,272,272,321,323,320,315,322,330,318,271.0,275.0,266.0,258.0,277.0,286.0,261.0,52.3,52.6,42.9,56.3,52.2,52.0,56.2,54.3,49.44
1,P002,9/10,54.7,55.1,54.2,28.1,56.5,56.9,56.0,55.4,58.1,55.8,56.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,322,321,322,244,331,331,331,316,336,335,...,58.1,50.4,NaN,297,301,292,241,332,332,332,321,337,334,336,295.0,300.0,290.0,273.0,300.0,322.0,NaN,52.8,54.1,44.6,61.6,54.8,59.1,60.9,58.7,57.00
2,P003,6/10,50.5,50.5,50.5,38.0,63.7,54.6,62.7,63.3,64.7,53.8,62.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,319,318,320,246,329,329,330,322,328,336,...,47.7,49.2,42.6,287,290,284,250,329,330,329,319,331,338,329,286.0,291.0,281.0,271.0,287.0,312.0,274.0,44.1,50.0,43.9,52.5,51.5,50.8,42.9,43.6,40.60
3,P006,7/10,51.5,53.5,49.4,29.7,54.9,55.8,53.9,56.3,56.4,56.3,50.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,298,299,297,249,309,310,308,298,313,316,...,60.8,56.1,59.2,263,266,261,248,307,306,308,296,308,313,310,259.0,261.0,257.0,242.0,260.0,280.0,253.0,58.3,61.3,53.5,61.2,58.5,57.1,58.5,58.3,56.80


## V3

In [619]:
dp_v3_piv=df_pivot_dp[cols].xs('V3',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=list )

dp_v3_piv=dp_v3_piv.applymap(lambda x: x[0] if type(x)==list else x)

a_dp_v3=dp_v3_piv.xs('L eye',level=1,axis=1).xs('3.0',level=1,axis=1)
b_dp_v3=dp_v3_piv.xs('L eye',level=1,axis=1).xs('6.0',level=1,axis=1)
c_dp_v3=dp_v3_piv.xs('R eye',level=1,axis=1).xs('3.0',level=1,axis=1)
d_dp_v3=dp_v3_piv.xs('R eye',level=1,axis=1).xs('6.0',level=1,axis=1)
dp_v3=pd.concat([a_dp_v3[cols],b_dp_v3[cols],c_dp_v3[cols],d_dp_v3[cols]],axis=1)
dp_v3=dp_v3.reset_index()
dp_v3

,pat,QUALITY,depth_whole,depth_sup hemi,depth_inf hemi,depth_fovea,depth_parafovea,depth_para_sup hemi,depth_parai_inf hemi,depth_para temp,depth_para_sup,depth_para_nasal,depth_para_inf,depth_perifovea,depth_peri_sup hemi,depth_peri_inf hemi,depth_peri_tempo,depth_peri_sup,depth_peri_nasal,depth_peri_inf,thick_whole,thick_sup hemi,thick_inf hemi,thick_fovea,thick_parafovea,thick_para_sup hemi,thick_parai_inf hemi,thick_para temp,thick_para_sup,thick_para_nasal,...,depth_peri_sup,depth_peri_nasal,depth_peri_inf,thick_whole,thick_sup hemi,thick_inf hemi,thick_fovea,thick_parafovea,thick_para_sup hemi,thick_parai_inf hemi,thick_para temp,thick_para_sup,thick_para_nasal,thick_para_inf,thick_perifovea,thick_peri_sup hemi,thick_peri_inf hemi,thick_peri_tempo,thick_peri_sup,thick_peri_nasal,thick_peri_inf,grid_v_d1,grid_v_d2,grid_v_d3,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9
0,P001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59.6,53.2,59.7,276,279,273,278,325,326,324,319,325,334,323,272.0,276.0,268.0,260.0,277.0,287.0,263.0,57.0,59.2,52.9,58.8,56.0,54.44,59.9,59.4,52.5
1,P002,9/10,54.9,55.4,54.4,30.4,573.0,57.9,56.7,56.6,58.8,56.8,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,317.0,316.0,318.0,241.0,327.0,326.0,327.0,313.0,332.0,331.0,...,51.5,54.6,55.8,295,299,290,243,331,332,329,319,337,332,334,293.0,298.0,289.0,272.0,297.0,320.0,285.0,48.7,52.9,47.0,56.8,52.8,55.60,56.9,54.5,51.0
2,P003,9/10,56.3,56.3,56.4,36.3,58.4,58.7,58.0,57.7,58.6,58.5,58.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,320.0,321.0,320.0,247.0,331.0,331.0,331.0,322.0,333.0,339.0,...,58.7,59.2,584.0,286,289,283,246,329,329,329,317,331,337,331,285.0,289.0,280.0,267.0,285.0,313.0,274.0,55.9,594.0,55.6,59.9,57.5,59.20,55.0,57.7,55.5
3,P006,7/10,53.5,52.7,54.4,30.8,56.7,56.1,57.3,58.0,55.6,56.5,56.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,302.0,303.0,301.0,249.0,312.0,313.0,311.0,300.0,316.0,321.0,...,49.9,44.0,47.2,264,268,260,248,309,308,309,299,310,315,311,261.0,264.0,258.0,247.0,263.0,282.0,254.0,41.5,49.3,43.1,49.3,50.9,473.00,46.3,45.2,45.9


# DENSITY SUP

## V2

In [620]:
sup_v2_piv=df_pivot_sup[cols].xs('V2',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=list )

sup_v2_piv=sup_v2_piv.applymap(lambda x: x[0] if type(x)==list else x)

a_sup_v2=sup_v2_piv.xs('L eye',level=1,axis=1).xs('3.0',level=1,axis=1)
b_sup_v2=sup_v2_piv.xs('L eye',level=1,axis=1).xs('6.0',level=1,axis=1)
c_sup_v2=sup_v2_piv.xs('R eye',level=1,axis=1).xs('3.0',level=1,axis=1)
d_sup_v2=sup_v2_piv.xs('R eye',level=1,axis=1).xs('6.0',level=1,axis=1)
sup_v2=pd.concat([a_sup_v2[cols],b_sup_v2[cols],c_sup_v2[cols],d_sup_v2[cols]],axis=1)
sup_v2=sup_v2.reset_index()
sup_v2

,pat,QUALITY,depth_whole,depth_sup hemi,depth_inf hemi,depth_fovea,depth_parafovea,depth_para_sup hemi,depth_parai_inf hemi,depth_para temp,depth_para_sup,depth_para_nasal,depth_para_inf,depth_perifovea,depth_peri_sup hemi,depth_peri_inf hemi,depth_peri_tempo,depth_peri_sup,depth_peri_nasal,depth_peri_inf,thick_whole,thick_sup hemi,thick_inf hemi,thick_fovea,thick_parafovea,thick_para_sup hemi,thick_parai_inf hemi,thick_para temp,thick_para_sup,thick_para_nasal,...,depth_peri_sup,depth_peri_nasal,depth_peri_inf,thick_whole,thick_sup hemi,thick_inf hemi,thick_fovea,thick_parafovea,thick_para_sup hemi,thick_parai_inf hemi,thick_para temp,thick_para_sup,thick_para_nasal,thick_para_inf,thick_perifovea,thick_peri_sup hemi,thick_peri_inf hemi,thick_peri_tempo,thick_peri_sup,thick_peri_nasal,thick_peri_inf,grid_v_d1,grid_v_d2,grid_v_d3,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9
0,P001,8/10,46.0,46.0,46.1,153.0,49.9,49.6,50.2,48.9,51.4,48.4,50.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307,308,306,267,319,321,316,309,323,324,...,48.7,53.6,48.9,276,280,272,272,321,323,320,315,322,330,318,271.0,275.0,266.0,258.0,277.0,286.0,261.0,42.3,49.8,54.2,48.4,45.2,54.2,47.7,49.44,52.2
1,P002,9/10,453.0,45.6,45.0,154.0,48.0,48.4,47.5,44.9,49.7,48.5,48.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,322,321,322,244,331,331,331,316,336,335,...,54.7,55.5,49.9,297,301,292,241,332,332,332,321,337,334,336,295.0,300.0,290.0,273.0,300.0,322.0,285.0,45.4,54.3,55.6,473.0,42.7,54.2,473.0,49.10,54.4
2,P003,6/10,36.7,37.6,35.9,15.9,39.0,40.2,37.8,37.8,41.9,37.7,38.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,319,318,320,246,329,329,330,322,328,336,...,47.7,50.6,453.0,287,290,284,250,329,330,329,319,331,338,329,286.0,291.0,281.0,271.0,287.0,312.0,274.0,38.6,49.6,54.4,45.9,43.9,48.2,39.0,44.00,53.1
3,P006,7/10,46.3,48.1,44.5,12.3,50.6,51.6,49.6,49.1,54.6,49.5,49.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,298,299,297,249,309,310,308,298,313,316,...,54.3,573.0,52.2,263,266,261,248,307,306,308,296,308,313,310,259.0,261.0,257.0,242.0,260.0,280.0,253.0,47.6,55.0,58.7,49.8,49.6,56.4,46.8,52.30,56.5


## V3

In [621]:
sup_v3_piv=df_pivot_sup[cols].xs('V3',level=1).pivot_table(index='pat',
                                           columns=['eye','depth'],
                                           aggfunc=list )

sup_v3_piv=sup_v3_piv.applymap(lambda x: x[0] if type(x)==list else x)

a_sup_v3=sup_v3_piv.xs('L eye',level=1,axis=1).xs('3.0',level=1,axis=1)
b_sup_v3=sup_v3_piv.xs('L eye',level=1,axis=1).xs('6.0',level=1,axis=1)
c_sup_v3=sup_v3_piv.xs('R eye',level=1,axis=1).xs('3.0',level=1,axis=1)
d_sup_v3=sup_v3_piv.xs('R eye',level=1,axis=1).xs('6.0',level=1,axis=1)
sup_v3=pd.concat([a_sup_v3[cols],b_sup_v3[cols],c_sup_v3[cols],d_sup_v3[cols]],axis=1)
sup_v3=sup_v3.reset_index()
sup_v3

,pat,QUALITY,depth_whole,depth_sup hemi,depth_inf hemi,depth_fovea,depth_parafovea,depth_para_sup hemi,depth_parai_inf hemi,depth_para temp,depth_para_sup,depth_para_nasal,depth_para_inf,depth_perifovea,depth_peri_sup hemi,depth_peri_inf hemi,depth_peri_tempo,depth_peri_sup,depth_peri_nasal,depth_peri_inf,thick_whole,thick_sup hemi,thick_inf hemi,thick_fovea,thick_parafovea,thick_para_sup hemi,thick_parai_inf hemi,thick_para temp,thick_para_sup,thick_para_nasal,...,depth_peri_sup,depth_peri_nasal,depth_peri_inf,thick_whole,thick_sup hemi,thick_inf hemi,thick_fovea,thick_parafovea,thick_para_sup hemi,thick_parai_inf hemi,thick_para temp,thick_para_sup,thick_para_nasal,thick_para_inf,thick_perifovea,thick_peri_sup hemi,thick_peri_inf hemi,thick_peri_tempo,thick_peri_sup,thick_peri_nasal,thick_peri_inf,grid_v_d1,grid_v_d2,grid_v_d3,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9
0,P001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50.5,52.9,49.9,276,279,273,278,325,326,324,319,325,334,323,272.0,276.0,268.0,260.0,277.0,287.0,263.0,43.3,51.0,53.1,44.8,46.0,53.0,46.8,50.3,51.9
1,P002,9/10,47.2,47.3,47.1,14.5,50.5,50.9,50.1,48.5,51.5,50.8,513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,317.0,316.0,318.0,241.0,327.0,326.0,327.0,313.0,332.0,331.0,...,54.8,57.7,53.2,295,299,290,243,331,332,329,319,337,332,334,293.0,298.0,289.0,272.0,297.0,320.0,285.0,44.9,56.3,59.2,48.7,45.7,55.5,50.6,52.5,57.5
2,P003,9/10,48.2,48.1,48.3,18.9,51.2,51.4,51.1,49.4,52.9,51.2,51.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,320.0,321.0,320.0,247.0,331.0,331.0,331.0,322.0,333.0,339.0,...,50.5,53.9,50.0,286,289,283,246,329,329,329,317,331,337,331,285.0,289.0,280.0,267.0,285.0,313.0,274.0,42.1,51.9,56.2,47.7,47.2,52.5,44.1,48.8,55.4
3,P006,7/10,47.8,48.3,47.2,12.3,51.5,51.9,51.0,49.3,54.4,50.7,51.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,302.0,303.0,301.0,249.0,312.0,313.0,311.0,300.0,316.0,321.0,...,46.1,48.7,43.8,264,268,260,248,309,308,309,299,310,315,311,261.0,264.0,258.0,247.0,263.0,282.0,254.0,3644.0,46.6,50.2,40.5,40.2,46.8,38.1,41.8,49.0


## ALL COLLATED V2

In [675]:
collated_v2=pd.concat([
    a_sup_v2,a_dp_v2,a_faz_v2,
    b_sup_v2,b_dp_v2,b_faz_v2,
    c_sup_v2,c_dp_v2,c_faz_v2,
    d_sup_v2,d_dp_v2,d_faz_v2,
                      ],axis=1,)
collated_v2=collated_v2.reset_index()
collated_v2

,pat,QUALITY,depth_fovea,depth_inf hemi,depth_para temp,depth_para_inf,depth_para_nasal,depth_para_sup,depth_para_sup hemi,depth_parafovea,depth_parai_inf hemi,depth_peri_inf,depth_peri_inf hemi,depth_peri_nasal,depth_peri_sup,depth_peri_sup hemi,depth_peri_tempo,depth_perifovea,depth_sup hemi,depth_whole,grid_v_d1,grid_v_d2,grid_v_d3,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9,thick_fovea,...,grid_v_d2,grid_v_d3,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9,thick_fovea,thick_inf hemi,thick_para temp,thick_para_inf,thick_para_nasal,thick_para_sup,thick_para_sup hemi,thick_parafovea,thick_parai_inf hemi,thick_peri_inf,thick_peri_inf hemi,thick_peri_nasal,thick_peri_sup,thick_peri_sup hemi,thick_peri_tempo,thick_perifovea,thick_sup hemi,thick_whole,QUALITY,area (mm2),perim (mm),FD
0,P001,8/10,153.0,46.1,48.9,50.9,48.4,51.4,49.6,49.9,50.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,46.0,49.8,50.44,46.4,48.80,23.1,48.8,49.9,49.9,47.0,267,...,52.6,42.9,56.3,52.2,52.0,56.2,54.3,49.44,272,272,315,318,330,322,323,321,320,261.0,266.0,286.0,277.0,275.0,258.0,271.0,280,276,9/10,0.209,1.741,52.02
1,P002,9/10,154.0,45.0,44.9,48.8,48.5,49.7,48.4,48.0,47.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.6,453.0,48.7,48.60,48.6,48.60,22.1,42.7,51.7,48.9,47.4,244,...,54.1,44.6,61.6,54.8,59.1,60.9,58.7,57.00,241,292,321,336,334,337,332,332,332,NaN,290.0,322.0,300.0,300.0,273.0,295.0,301,297,9/10,-0.380,2.347,55.74
2,P003,6/10,15.9,35.9,37.8,38.6,37.7,41.9,40.2,39.0,37.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.6,36.7,38.0,42.80,38.2,37.20,21.0,40.0,39.9,40.3,32.6,246,...,50.0,43.9,52.5,51.5,50.8,42.9,43.6,40.60,250,284,319,329,338,331,330,329,329,274.0,281.0,312.0,287.0,291.0,271.0,286.0,290,287,8/10,0.254,1.911,53.52
3,P006,7/10,12.3,44.5,49.1,49.2,49.5,54.6,51.6,50.6,49.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.1,46.3,54.8,54.90,48.9,47.44,20.0,48.0,48.2,48.3,46.0,249,...,61.3,53.5,61.2,58.5,57.1,58.5,58.3,56.80,248,261,296,310,313,308,306,307,308,253.0,257.0,280.0,260.0,261.0,242.0,259.0,266,263,8/10,0.279,2.043,56.55


## ALL COLLATED V3

In [676]:
collated_v3=pd.concat([
    a_sup_v3,a_dp_v3,a_faz_v3,
    b_sup_v3,b_dp_v3,b_faz_v3,
    c_sup_v3,c_dp_v3,c_faz_v3,
    d_sup_v3,d_dp_v3,d_faz_v3,
                      
                      ],axis=1,)
collated_v3=collated_v3.reset_index()
collated_v3

,pat,QUALITY,depth_fovea,depth_inf hemi,depth_para temp,depth_para_inf,depth_para_nasal,depth_para_sup,depth_para_sup hemi,depth_parafovea,depth_parai_inf hemi,depth_peri_inf,depth_peri_inf hemi,depth_peri_nasal,depth_peri_sup,depth_peri_sup hemi,depth_peri_tempo,depth_perifovea,depth_sup hemi,depth_whole,grid_v_d1,grid_v_d2,grid_v_d3,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9,thick_fovea,...,grid_v_d2,grid_v_d3,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9,thick_fovea,thick_inf hemi,thick_para temp,thick_para_inf,thick_para_nasal,thick_para_sup,thick_para_sup hemi,thick_parafovea,thick_parai_inf hemi,thick_peri_inf,thick_peri_inf hemi,thick_peri_nasal,thick_peri_sup,thick_peri_sup hemi,thick_peri_tempo,thick_perifovea,thick_sup hemi,thick_whole,QUALITY,area (mm2),perim (mm),FD
0,P001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59.2,52.9,58.8,56.0,54.44,59.9,59.4,52.5,278,273,319,323,334,325,326,325,324,263.0,268.0,287.0,277.0,276.0,260.0,272.0,279,276,8/10,0.213,1.750,54.65
1,P002,9/10,14.5,47.1,48.5,513.0,50.8,51.5,50.9,50.5,50.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.3,47.2,51.7,50.9,49.7,51.2,21.9,453.0,533.0,51.6,48.6,241.0,...,52.9,47.0,56.8,52.8,55.60,56.9,54.5,51.0,243,290,319,334,332,337,332,331,329,285.0,289.0,320.0,297.0,298.0,272.0,293.0,299,295,9/10,-0.396,2.407,57.54
2,P003,9/10,18.9,48.3,49.4,51.5,51.2,52.9,51.4,51.2,51.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.1,48.2,51.5,51.7,50.6,50.5,24.5,50.1,53.5,51.8,49.1,247.0,...,594.0,55.6,59.9,57.5,59.20,55.0,57.7,55.5,246,283,317,331,337,331,329,329,329,274.0,280.0,313.0,285.0,289.0,267.0,285.0,289,286,9/10,0.253,1.907,57.48
3,P006,7/10,12.3,47.2,49.3,51.4,50.7,54.4,51.9,51.5,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.3,47.8,52.9,53.0,51.8,504.0,21.5,47.3,55.1,50.5,473.0,249.0,...,49.3,43.1,49.3,50.9,473.00,46.3,45.2,45.9,248,260,299,311,315,310,308,309,309,254.0,258.0,282.0,263.0,264.0,247.0,261.0,268,264,7/10,0.282,2.035,51.40


# EXCEL FILLING 

In [715]:
from openpyxl.utils.dataframe import dataframe_to_rows
wb=op.load_workbook('../data/NOPAIN TRIAL OCTA DATA EXTRACTION.xlsx')

In [716]:
sheet_names=wb.sheetnames
sheet_names

['V2 COLLATED',
 'V3 COLLATED',
 'V2 DENSITY DP DATA ',
 'V3 DENSITY DP DATA',
 'V2 DENSITY SUP DATA',
 'V3 DENSITY SUP DATA',
 'V2 FAZ',
 'V3 FAZ']

# WRITING

In [717]:
###################################################################### V2_COLLATED
wb.active=wb['V2 COLLATED']
ws=wb.active
rows = dataframe_to_rows(collated_v2,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)
###################################################################### V3_COLLATED
wb.active=wb['V3 COLLATED']
ws=wb.active
rows = dataframe_to_rows(collated_v3,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)
            
###################################################################### V2_DENSITY_DP
wb.active=wb[ 'V2 DENSITY DP DATA ']
ws=wb.active
rows = dataframe_to_rows(dp_v2,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)

###################################################################### V3_DENSITY_DP
wb.active=wb[ 'V3 DENSITY DP DATA']
ws=wb.active
rows = dataframe_to_rows(dp_v3,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)
###################################################################### V2_DENSITY_SUP
wb.active=wb[ 'V2 DENSITY SUP DATA']
ws=wb.active
rows = dataframe_to_rows(sup_v2,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)
###################################################################### V3_DENSITY_SUP
wb.active=wb[ 'V3 DENSITY SUP DATA']
ws=wb.active
rows = dataframe_to_rows(sup_v3,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)
            
###################################################################### V2_FAZ
wb.active=wb[ 'V2 FAZ']
ws=wb.active
rows = dataframe_to_rows(faz_v2,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)
###################################################################### V3_FAZ
wb.active=wb[ 'V3 FAZ']
ws=wb.active
rows = dataframe_to_rows(faz_v3,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)


<Cell 'V2 COLLATED'.A4>

<Cell 'V2 COLLATED'.B4>

<Cell 'V2 COLLATED'.C4>

<Cell 'V2 COLLATED'.D4>

<Cell 'V2 COLLATED'.E4>

...

<Cell 'V3 FAZ'.M7>

<Cell 'V3 FAZ'.N7>

<Cell 'V3 FAZ'.O7>

<Cell 'V3 FAZ'.P7>

<Cell 'V3 FAZ'.Q7>

In [719]:
wb.save("OCTA_EXTRACTION_FILLED.xlsx")